In [ ]:
import alsm
from matplotlib import pyplot as plt
import matplotlib as mpl
import numpy as np
from scipy import stats


mpl.rcParams['figure.dpi'] = 144
SEED = 3

In [ ]:
np.random.seed(SEED)

fig, (ax1, ax2) = plt.subplots(1, 2)

ax = ax1
num_dims = 2
weighted = False
# Evaluate the kernel over a range of scales and separations.
ax.set_yscale('log')
dx = np.linspace(0, 3, 100)
delta = np.stack([dx, *np.zeros((num_dims - 1, 100))]).T
scale = np.logspace(-2, 1, 101)

mean = alsm.evaluate_mean(np.zeros(num_dims), delta[:, None], scale, scale, 1).T
mappable = ax.pcolormesh(*np.meshgrid(dx, scale), mean, shading='auto')
cb = fig.colorbar(mappable, ax=ax, location='top')
cb.set_label('Kernel $\lambda$')

levels = [0.05, 0.15, 0.3, 0.6, 0.9]
cs = ax.contour(dx, scale, mean, colors='w', levels=levels, linestyles='--')
plt.clabel(cs)

# Plot the maximum last.
ax.autoscale(False)
argmax_delta = (num_dims * (1 + 2 * scale ** 2)) ** .5
ax.plot(argmax_delta, scale, color='C1')

ax.set_ylabel(r'Scale $\sigma$')
ax.set_xlabel(r'Separation $\delta$')
ax.text(0.05, 0.95, '(a)', va='top', transform=ax.transAxes, color='w')

# Show simulated data.
ax = ax2
num_samples = 20000
group_sizes = np.asarray([10, 15])
means = np.asarray([[0, 0], [1, 0]])
scale = 5
x, y = [np.random.normal(mean, scale, (num_samples, size, 2)) for mean, size 
        in zip(means, group_sizes)]
rate = alsm.evaluate_kernel(x[:, :, None], y[:, None], 1)
if weighted:
    agg = np.random.poisson(rate.sum(axis=(1, 2)))
else:
    agg = np.random.binomial(1, rate).sum(axis=(1, 2))
counts = np.bincount(agg) / agg.size
ax.bar(np.arange(counts.size), counts, color='silver', width=1)

# Evaluate properties of the negative binomial approximation.
mean = alsm.evaluate_aggregate_mean(*means, scale, scale, 1, *group_sizes)
var = alsm.evaluate_aggregate_var(*means, scale, scale, 1, *group_sizes, weighted)
if weighted:
    dist = stats.nbinom(*alsm.evaluate_negative_binomial_np(mean, var))
else:
    trials = np.prod(group_sizes)
    dist = stats.betabinom(trials, *alsm.evaluate_beta_binomial_ab(trials, mean, var))
lin = np.arange(agg.min(), agg.max() + 1)
ax.plot(lin, dist.pmf(lin))

ax.set_xlabel('Aggregate connections $Y_{ab}$')
ax.set_ylabel('$P(Y_{ab})$')
ax.yaxis.major.formatter.useMathText = True
ax.yaxis.major.formatter.set_powerlimits((0, 0))
ax.text(0.05, 0.95, '(b)', va='top', transform=ax.transAxes)
lines = [
    r'$\delta=1$',
    fr'$\sigma={scale}$',
    f'$n_a={group_sizes[0]}$',
    f'$n_b={group_sizes[1]}$',
]
ax.text(0.95, 0.95, '\n'.join(lines), va='top', ha='right', transform=ax.transAxes)

fig.tight_layout()
fig.savefig('../workspace/kernel.pdf')